In [1]:
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline
import spacy

# --- 1. Load All Models (This happens only once when the app starts) ---
print("Loading models... This may take a few minutes.")

# Load the NLP model for keyword extraction
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading 'en_core_web_sm' spacy model...")
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

# Load the AI Image Generation model (Stable Diffusion)
model_id = "runwayml/stable-diffusion-v1-5"
# Use float16 for less memory usage.
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Move the model to the GPU if one is available (much faster)
if torch.cuda.is_available():
    pipe = pipe.to("cuda")
else:
    print("Warning: CUDA (GPU) not available. Running on CPU, which will be very slow.")

print("Models loaded successfully!")

# --- 2. Define the Core Logic Function ---

def story_to_illustration(story_line, art_style):
    """
    This function takes a line of a story and an art style,
    and returns a generated image.
    """
    print(f"Received story: '{story_line}', Style: '{art_style}'")

    # Part A: Extract Keywords using spaCy
    doc = nlp(story_line)
    keywords = []
    for token in doc:
        # Focus on the main subjects and objects
        if token.pos_ in ["NOUN", "PROPN"]:
            subject = ""
            # Include adjectives describing the noun
            for child in token.children:
                if child.pos_ == "ADJ":
                    subject += child.text + " "
            subject += token.text
            keywords.append(subject)

    if not keywords:
        # If no specific nouns are found, use the whole text as a fallback
        keywords_str = story_line
    else:
        keywords_str = ", ".join(keywords)

    print(f"Extracted Keywords: '{keywords_str}'")

    # Part B: Construct a Detailed Prompt for the AI
    # Prompt engineering is key for good results!
    prompt = f"award-winning children's storybook illustration of {keywords_str}, {art_style}, vibrant colors, detailed, beautiful, whimsical"
    negative_prompt = "blurry, ugly, deformed, noisy, text, watermark, signature, person, man, woman"

    print(f"Final Prompt: '{prompt}'")

    # Part C: Generate the Image
    # This is the main AI call
    image = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=50,
        guidance_scale=7.5
    ).images[0]

    return image

# --- 3. Create and Launch the Gradio Web Interface ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎨 AI Storybook Illustrator 🖌️")
    gr.Markdown("Enter a line from a story, choose an art style, and the AI will create an illustration for you!")

    with gr.Row():
        with gr.Column(scale=2):
            story_input = gr.Textbox(
                label="Story Line",
                placeholder="e.g., A brave little fox explores a magical forest at night"
            )
            style_input = gr.Dropdown(
                label="Artistic Style",
                choices=[
                    "Vibrant watercolor style",
                    "Cute claymation style",
                    "Japanese anime movie style",
                    "Retro 8-bit pixel art",
                    "Indian Madhubani folk art",
                    "Whimsical cartoon style"
                ],
                value="Vibrant watercolor style"
            )
            submit_btn = gr.Button("Generate Illustration", variant="primary")

        with gr.Column(scale=3):
            image_output = gr.Image(label="Generated Illustration")

    submit_btn.click(
        fn=story_to_illustration,
        inputs=[story_input, style_input],
        outputs=image_output
    )

# Launch the app!
demo.launch()

Loading models... This may take a few minutes.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Models loaded successfully!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f8c5ba2a79f273182a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
